In [87]:
from nltk.grammar import DependencyGrammar
from nltk.parse import (
    DependencyGraph,
    ProjectiveDependencyParser,
    NonprojectiveDependencyParser,
)
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

Где прочитать про алгоритмы если вдруг интересно:

работа про DRAGNN (гугловский парсер):

работа про UDPIPE (популярный парсер):

работа с хорошим Related works, из которого можно понять примерный раклад и куда дальше копать:

Какие есть корпуса:
    
Сайт Universal dependencies приводит ссылки на доступные корпуса (там уже много языков)

Для стандартный - Syntagrus
Скоро должен выйти большой корпус Taiga (сейчас доступна небольшая часть)

На них можно обучать и тестрировать модели.

Большой размеченный UDPipe корпус можно взять тут (острожно 27 гб в сжатом виде)

Скорее всего сами вы обучать ничего не будете, а будет использовать готовые инструменты.

Два самых популярных:
    Syntaxnet (DRAGNN) - от гугла
    UDPipe
    
SyntaxNet активно развивается, но его достаточно трудоемко ставить 
UdPipe работает похуже, но установить его легче

Если вам нужен английский, то удобнее всего разбирать предложения через spacy.



Посмотрим, как выглядит разобранное предложение. Я взял 50 тыс предложений из корпуса. Там много мусора, но на больших количествах это не сильно мешает.

In [114]:
sentences = open('data/sentences_100000.txt').read().splitlines()

In [115]:
trees = open('data/trees_100000.txt').read().split('\n\n')

In [29]:
# Пример предложения
sentences[1]

'приливы, нарушения сна, психологические расстройства и сухость во'

In [164]:
# Как выглядит его дерево
[x.replace('\t', ' ') for x in trees[2].split('\n')]

['1 Когда когда SCONJ _ _ 14 mark _ _',
 '2 у у ADP _ _ 3 case _ _',
 '3 вас вы PRON _ Case=Gen|Number=Plur|Person=2 4 obl _ _',
 '4 появляется появляться VERB _ Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Mid 14 advcl _ _',
 '5 нагрузка нагрузка NOUN _ Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing 4 nsubj _ SpaceAfter=No',
 '6 , , PUNCT _ _ 5 punct _ _',
 '7 которую который PRON _ Case=Acc 9 obj _ _',
 '8 нужно нужный ADJ _ Degree=Pos|Gender=Neut|Number=Sing|Variant=Short 5 acl:relcl _ _',
 '9 оптимизировать оптимизировать VERB _ Aspect=Imp|VerbForm=Inf|Voice=Act 8 nsubj _ _',
 '10 — — PUNCT _ _ 11 punct _ _',
 '11 php php X _ Foreign=Yes 9 flat:foreign _ SpaceAfter=No',
 '12 / / PUNCT _ _ 11 punct _ SpaceAfter=No',
 '13 ruby/python ruby/pythoн-лат X _ Foreign=Yes 11 flat:foreign _ _',
 '14 становятся становиться VERB _ Aspect=Imp|Mood=Ind|Number=Plur|Person=3|Tense=Pres|VerbForm=Fin|Voice=Mid 0 root _ _',
 '15 убогими убогий ADJ _ Case=Ins|Degree=Pos|Number=

In [ ]:
В nltk есть удобный класс для загрузки таки

In [146]:
d = DependencyGraph(trees[10])

In [123]:
d.nodes

defaultdict(<function nltk.parse.dependencygraph.DependencyGraph.__init__.<locals>.<lambda>>,
            {0: {'address': 0,
              'ctag': 'TOP',
              'deps': defaultdict(list, {'ROOT': [], 'root': [3]}),
              'feats': None,
              'head': None,
              'lemma': None,
              'rel': None,
              'tag': 'TOP',
              'word': None},
             1: {'address': 1,
              'ctag': 'ADV',
              'deps': defaultdict(list, {}),
              'feats': 'Degree=Pos',
              'head': 2,
              'lemma': 'абсолютно',
              'rel': 'advmod',
              'tag': '_',
              'word': 'Абсолютно'},
             2: {'address': 2,
              'ctag': 'ADJ',
              'deps': defaultdict(list, {'advmod': [1]}),
              'feats': 'Animacy=Inan|Case=Acc|Degree=Pos|Number=Plur',
              'head': 3,
              'lemma': 'разный',
              'rel': 'amod',
              'tag': '_',
          

In [141]:
def get_subtree(nodes, node, restrictions=[]):
    
    
    if not nodes[node]['deps']:
        return [node]
    else:
        
        
        return [node] + [get_subtree(nodes, dep) for rel in nodes[node]['deps'] if rel != 'punct'\
                         and not (restrictions and rel not in restrictions) \
                         for dep in nodes[node]['deps'][rel]]
# flatten = lambda l: [item for sublist in l for item in sublist]
def flatten(l):
    flat = []
    for el in l:
        if not isinstance(el, list):
            flat.append(el)
        else:
            flat += flatten(el)
    return flat

def get_words(nodes, node, level='word',restrictions=[]):
    ns = (get_subtree(nodes, node, restrictions))
    words = []
    for i in ns:
        if isinstance(i, list):
            ws = sorted(flatten(i))
            words.append(' '.join([nodes[j][level] for j in ws]))
        else:
            words.append(nodes[i][level])
    return words

def get_corpus_stat_for_word(trees, word, level='word', rel=[]):
    stat = Counter()
    heads = Counter()
    for tree in trees:
        try:
            nodes = DependencyGraph(tree).nodes
        except:
            continue
        
        for i_word in nodes:
            if nodes[i_word][level] == word:
                sub = get_words(nodes, i_word, level, rel)
                head = get_head(nodes, i_word)
                heads.update([head])
                stat.update(sub)
    
    w_stat = stat[word]
    del stat[word]
    
    return w_stat, stat, heads

def get_head(nodes, node, level='word'):
    head = nodes[node]['head']
    w_head = nodes[head][level]
    rel = nodes[node]['rel']
    
    return w_head, rel, nodes[node][level]

In [150]:
d.nodes[0]['deps']['root']

[2]

In [151]:
get_words(d.nodes, d.nodes[0]['deps']['root'][0])

['уменьшили', 'Вы', 'настолько что не можете увидеть флажки', 'масштаб']

In [127]:

get_head(d.nodes, 1)

('разные', 'advmod')

In [53]:
restrictions = ['conj']
rel = 'conj'
not (restrictions and rel not in restrictions)

True

In [143]:
get_corpus_stat_for_word(trees,'аналог', 'lemma', [])

(69,
 Counter({'1379.1112110': 1,
          '236 1007025': 1,
          '273': 1,
          'Am': 1,
          'Openoffice': 1,
          '«Метелицы»': 1,
          'Мексидола': 1,
          'Мексидола тут': 1,
          'Распылитель': 1,
          'аргинин': 1,
          'без': 1,
          'бесплатный': 1,
          'более легкий': 1,
          'быть': 1,
          'в мир': 1,
          'в один таблицеН': 1,
          'включайть в себя достойный замена Word и Excel': 1,
          'вместо': 1,
          'выше': 1,
          'данный препарат': 1,
          'дешевый': 1,
          'жидкий гран': 1,
          'западный': 2,
          'зарубежный': 2,
          'зодиакальный': 1,
          'и': 6,
          'и новинка': 1,
          'из 1-й опытный группа': 1,
          'из контрольный группа': 1,
          'из линия LeBaron': 1,
          'из-за рубеж': 1,
          'изготавливать на заказ': 1,
          'или': 2,
          'иметься': 1,
          'иметься в каждый округ': 1,
          '

In [153]:
rels = set()
poss = set()
for tree in trees:
    try:
        labels = [x.split('\t')[7] for x in tree.split('\n')]
        pos = [x.split('\t')[3] for x in tree.split('\n')]
    except Exception as e:
        print(e)
        continue
    rels.update(labels)
    poss.update(pos)

list index out of range


In [154]:
rels

{'acl',
 'acl:relcl',
 'advcl',
 'advmod',
 'amod',
 'appos',
 'aux',
 'aux:pass',
 'case',
 'cc',
 'ccomp',
 'compound',
 'conj',
 'cop',
 'dep',
 'discourse',
 'expl',
 'fixed',
 'flat',
 'flat:foreign',
 'flat:name',
 'iobj',
 'mark',
 'nmod',
 'nsubj',
 'nsubj:pass',
 'nummod',
 'nummod:entity',
 'nummod:gov',
 'obj',
 'obl',
 'obl:agent',
 'orphan',
 'parataxis',
 'punct',
 'root',
 'xcomp'}

In [152]:
letters = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRST'
digits = '0123456789UVWXYZ'

rel2id = {rel:letters[i] for i, rel in enumerate(rels)}


NameError: name 'rels' is not defined

In [ ]:
# Функции для превращения деревьев в формат, необходимый для tree adit distance (Zhu)
sp = []
def transform(tree):
    global sp
    sp = reshape(tree.nodes)
    return eval(parse(sp[0]))

def transform_raw(tree):
    global sp
    sp = reshape(tree.nodes)
    return parse(sp[0])

def reshape(d):
    sp = [(d[x]['rel'], [j for i in d[x]['deps'] for j in d[x]['deps'][i]]) for x in range(len(d))]
    return sp

def parse(tree):
    if len(tree[1]) == 0:
        return 'Node("{}", [])'.format(tree[0])
    else:
        return 'Node("{}", ['.format(tree[0]) + ','.join([parse(sp[x]) for x in tree[1]]) + '])'

In [155]:
import zss
import re

def subtree(node, tree):
    if not tree[node]['deps']:
        word = tree[node]['word']
        if not word:
            word = 'UNK'
        pos = tree[node]['ctag']
        rel = tree[node]['rel']
        label = 'WORD=#{}#,POS=#{}#,REL=#{}#'.format(word, pos, rel)
        return "WNode('{}', [])".format(label)
    else:
        sub = []
        for dep in tree[node]['deps']:
            for n in tree[node]['deps'][dep]:
                sub.append(subtree(n, tree))
        
        if not tree[node]['rel']:
            if tree[node]['ctag'] == 'TOP':
                name = 'TOP'
                word = 'TOP'
                if not word:
                    word = 'UNK'
                pos = 'TOP'
                rel = 'TOP'
            else:
                print('Node numeber - {} is None and not TOP!'.format(node))
                raise KeyError
        else:    
            word = tree[node]['word']
            pos = tree[node]['ctag']
            rel = tree[node]['rel']
        label = 'WORD=#{}#,POS=#{}#,REL=#{}#'.format(word, pos, rel)
            
        return "WNode('{}', [{}])".format(label , ', '.join(sub))

# def to_zss(tree):
#     top = "Node('TOP', [{}])"
#     deps = []
#     for dep in tree.nodes[0]['deps']:
#         for 

def weird_dist(A, B):
    ma = re.search('WORD=#(.*?)#,POS=#(.*?)#,REL=#(.*?)#', A)
    mb = re.search('WORD=#(.*?)#,POS=#(.*?)#,REL=#(.*?)#', B)
    
    if not ma and mb:
        return 1
    elif ma and not mb:
        return 3
    elif not ma and not mb:
        print('NO')
        return 0
    
    word_a, pos_a, rel_a = ma.groups()
    word_b, pos_b, rel_b = mb.groups()
    dist = 0
    
    
    v_a = w2v[word_a] if word_a in w2v else None
    v_b = w2v[word_b] if word_b in w2v else None
    
    
    if type(v_a) != type(v_b):
        dist += 1
    
    else:
        if v_a is not None:
            if (1 - cosine(v_a, v_b)) < 0.7:
                dist += 1
        else:
            if word_a != word_b:
                dist += 1
    
    if pos_a != pos_b:
#         print(pos_a, pos_b)
        dist += 1
    
    if rel_a != rel_b:
#         print(rel_a, rel_b)
        dist += 1
    
    
    return dist

class WNode(object):

    def __init__(self, label, children=None):
        self.my_label = label
        self.my_children = children or list()

    @staticmethod
    def get_children(node):
        return node.my_children

    @staticmethod
    def get_label(node):
        return node.my_label

    def addkid(self, node, before=False):
        if before:  self.my_children.insert(0, node)
        else:   self.my_children.append(node)
        return self
    
    def __str__(self):
        s = "%d:%s" % (len(self.my_children), self.my_label)
        s = '\n'.join([s]+[str(c) for c in self.my_children])
        return s



In [156]:
import gensim

In [159]:
sents = []
for tree in trees:
    try:
        words = [x.split('\t')[1] for x in tree.split('\n')]
    except:
        continue
    sents.append(words)

In [160]:
sents[:10]

[['Естественно', 'цена', 'там', 'была', 'аховая', '.'],
 ['–', 'Может', '.', '.', '.'],
 ['Когда',
  'у',
  'вас',
  'появляется',
  'нагрузка',
  ',',
  'которую',
  'нужно',
  'оптимизировать',
  '—',
  'php',
  '/',
  'ruby/python',
  'становятся',
  'убогими',
  '.'],
 ['Турецкий',
  'сериал',
  'Серебро',
  '/',
  'Гюмюш',
  '/',
  'Gümüş',
  '61',
  'серия',
  'онлайн',
  'с',
  'русскими',
  'субтитрами'],
 ['Подхалимство',
  'и',
  'его',
  'неизбежное',
  'следствие',
  '-',
  'очковтирательство',
  'могут',
  '.'],
 ['См.',
  'похожие',
  'отели',
  'Belltown',
  'Inn',
  '798',
  'отзывовПоказать',
  'цены',
  '0.3',
  'км',
  'Hotel',
  'Five',
  '-',
  'A',
  'Piece',
  'of',
  'Pineapple',
  'Hospitality',
  '1',
  '064',
  'отзываПоказать',
  'цены',
  '0.6',
  'км',
  'The',
  'Edgewater',
  ',',
  'A',
  'Noble',
  'House',
  'Hotel',
  '3',
  '279',
  'отзывовПоказать',
  'цены',
  '0.3',
  'км',
  'Сиэтл',
  ':',
  'просмотреть',
  'все',
  'отели',
  'Интересное',
 